# ニューラルネットワークのビルド

In [2]:
import os

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
# デバイスの設定
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(f'using {device} device')

using mps device


In [5]:
# ニューラルネットワーククラス
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10))
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [14]:
# 使用するには，入力をモデルに与える
# このとき，forward処理もその他の処理とともに動作する
# forwardを直接コールする必要はない

X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(f"model output(raw):\n{logits.to('cpu')}")
pred_probab = nn.Softmax(dim=1)(logits)
print(f"model output(prob):\n{pred_probab}")
y_pred = pred_probab.argmax(1)
print(f'predicted class: {y_pred}')

model output(raw):
tensor([[-0.0861, -0.0546,  0.0531, -0.0399, -0.0550,  0.0069,  0.0154, -0.0364,
          0.0433,  0.0030]], grad_fn=<ToCopyBackward0>)
model output(prob):
tensor([[0.0931, 0.0960, 0.1070, 0.0974, 0.0960, 0.1021, 0.1030, 0.0978, 0.1059,
         0.1017]], device='mps:0', grad_fn=<SoftmaxBackward0>)
predicted class: tensor([2], device='mps:0')


# 少々深堀り

In [15]:
# 3つの画像を想定したミニバッチを用意
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [16]:
# 画像を1次元配列に
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [17]:
# 線形変換層の動作の確認
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [20]:
# reluの動作確認
print(f"before relu: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
# 0未満の値は0, それ以外はそのまま
print(f"after relu: {hidden1}\n\n")

before relu: tensor([[0.0000, 0.2374, 0.1187, 0.1020, 0.0000, 0.0000, 0.0096, 0.0000, 0.0515,
         0.0584, 0.1903, 0.1448, 0.0000, 0.0588, 0.0528, 0.0958, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.3059, 0.0266, 0.0000, 0.1480, 0.0000, 0.1438, 0.0000, 0.0000,
         0.0000, 0.1188, 0.3235, 0.0957, 0.0000, 0.1193, 0.0582, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.1874, 0.0386, 0.1225, 0.0857, 0.0977, 0.0000, 0.0000, 0.0669,
         0.5749, 0.0000, 0.1729, 0.0083, 0.0000, 0.3544, 0.0744, 0.0000, 0.0000,
         0.0000, 0.0000]], grad_fn=<ReluBackward0>)


after relu: tensor([[0.0000, 0.2374, 0.1187, 0.1020, 0.0000, 0.0000, 0.0096, 0.0000, 0.0515,
         0.0584, 0.1903, 0.1448, 0.0000, 0.0588, 0.0528, 0.0958, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.3059, 0.0266, 0.0000, 0.1480, 0.0000, 0.1438, 0.0000, 0.0000,
         0.0000, 0.1188, 0.3235, 0.0957, 0.0000, 0.1193, 0.0582, 0.0000, 0.0000,
         0.0000, 0.0000],
       

In [23]:
# 直列にスタック
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10))

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [31]:
# 出力を確率に変換
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.1371, 0.0837, 0.0840, 0.0765, 0.1049, 0.0939, 0.1163, 0.1057, 0.1151,
         0.0827],
        [0.1376, 0.0818, 0.0917, 0.0761, 0.1087, 0.0897, 0.1236, 0.1055, 0.0990,
         0.0862],
        [0.1493, 0.0803, 0.0806, 0.0791, 0.1041, 0.0949, 0.1137, 0.1050, 0.1091,
         0.0840]], grad_fn=<SoftmaxBackward0>)


In [33]:
# モデルパラメータの確認
# nn.Moduleでは，すべてのパラメータがトラックされている
for name, param in model.named_parameters():
    print(f"layer: {name} | size: {param.size()} | values: {param[:2]}")

layer: linear_relu_stack.0.weight | size: torch.Size([512, 784]) | values: tensor([[ 0.0009,  0.0163,  0.0018,  ..., -0.0030, -0.0268, -0.0057],
        [ 0.0273, -0.0257,  0.0185,  ..., -0.0059,  0.0055,  0.0165]],
       device='mps:0', grad_fn=<SliceBackward0>)
layer: linear_relu_stack.0.bias | size: torch.Size([512]) | values: tensor([-0.0290,  0.0228], device='mps:0', grad_fn=<SliceBackward0>)
layer: linear_relu_stack.2.weight | size: torch.Size([512, 512]) | values: tensor([[ 0.0074, -0.0231,  0.0285,  ..., -0.0431, -0.0194,  0.0387],
        [-0.0111,  0.0087, -0.0190,  ..., -0.0200, -0.0116, -0.0146]],
       device='mps:0', grad_fn=<SliceBackward0>)
layer: linear_relu_stack.2.bias | size: torch.Size([512]) | values: tensor([ 0.0209, -0.0028], device='mps:0', grad_fn=<SliceBackward0>)
layer: linear_relu_stack.4.weight | size: torch.Size([10, 512]) | values: tensor([[ 0.0371, -0.0340,  0.0074,  ...,  0.0238,  0.0164, -0.0314],
        [-0.0086,  0.0183,  0.0248,  ..., -0.0256,  